# Calculating dissolved volatile concentrations

The `calculate_dissolved_volatiles()` function calcutions the concentration of dissolved H$_2$O and CO$_2$ in the liquid at a given pressure-temperature condition and with a given H$_2$O-CO$_2$ fluid composition, defined as the mole fraction of H$_2$O in an H$_2$O-CO$_2$ fluid (XH$_2$O$^{fluid}$). The default MagmaSat model relies on the underlying functionatlity of MELTS, whose basic function is to calculate the equilibrium phase assemblage given the bulk composition of the system and pressure-temperature conditions. To calculate dissolved volatile concentrations thus requires computing the equilibrium state of a system at fixed pressure and temperature over a range of bulk volatile concentrations until a solution is found that satisfies the user defined fluid composition.

First, the function makes an initial guess at the appropriate bulk volatile concentrations by finding the minimum dissolved volatile concentrations in the liquid at saturation, while asserting that the weight fraction of H$_2$O/$\Sigma$volatiles in the system is equal to the user input mole fraction of H$_2$O/$\Sigma$volatiles in the fluid. This is done by increasing the H$_2$O and CO$_2$ concentrations appropriately until a fluid phase is stable. Once fluid saturation is determined, the code then performs directional, iterative, and progressively more refined searches, increasing the proportion of H$_2$O or CO$_2$ in the system if the mole fraction of H$_2$O calculated in the fluid is greater than or less than that defined by the user, respectively. Four iterative searches are performed; the precision of the match between the calculated and defined XH$_2$O$^{fluid}$ increases from 0.1 in the first iteration to 0.01, 0.001, and finally to 0.0001. Thus, the calculated dissolved volatile concentrations correspond to a system with XH$_2$O$^{fluid}$ within 0.0001 of the user defined value.

**Method structure:**<br>
> Single sample: `def calculate_dissolved_volatiles(self, sample, temperature, pressure, X_fluid=1, verbose=False).result`

>ExcelFile batch process: `def calculate_dissolved_volatiles(self, temperature, pressure, X_fluid=1, print_status=False)`

**Required inputs:**<br>
>`sample`: *Only for single-sample calculations.* The composition of a sample. A single sample may be passed as a dictionary of values, with compositions of oxides in wt%.

>`temperature`, `pressure`, and `X_fluid`: the temperature in $^{\circ}$C, the pressure in bars, and the mole fraction of H$_2$O in the H$_2$O-CO$_2$ fluid, XH$_2$O$^{fluid}$. Temperature and pressure of the sample or samples must be passed unless an ExcelFile object with a column for temperature and/or pressure is passed to `sample`. XH$_2$O$^{fluid}$ is optional, with a default value of 1 (pure H$_2$O fluid). If a numerical (float) value is passed for either temperature, pressure, or X_fluid, that will be the value used for one or all samples. If, alternatively, the user wishes to use temperature, pressure, and/or X_fluid information in their ExcelFile object, the title of the column containing temperature, pressure, or X_fluid data should be passed in quotes (as a string) to `temperature`,  `pressure`, and/or `X_fluid`, respectively. Note for batch calculations that if temperature, pressure, or XH$_2$O$^{fluid}$ information exists in the ExcelFile but a single numerical value is defined for one or both of these variables, both the original information plus the values used for the calculations will be returned.

**Optional inputs:**<br>
>`verbose`: *Only for single-sample calculations.* Default value is False. If set to True, additional parameters are returned in a dictionary: H$_2$O and CO$_2$ concentrations in the fluid in mole fraction, temperature, pressure, and proportion of the fluid in the system in wt%.

> `print_status`: *Only for ExcelFile batch calcualtions.* The default value is False. If True is passed, the progress of the calculation will be printed to the terminal. The user may desire to see the status of the calculation, as this particular function can be quite slow, averaging between 3-5 seconds per sample. #TODO test this more and verify average compute time.

**Calculated outputs:**<br>
>If a single sample is passed to `sample`, a dictionary with keys 'H2O' and 'CO2' corresponding to the calculated dissolved H$_2$O and CO$_2$ concentrations in the liquid is returned (plus additional variables 'temperature' in $^{\circ}$C, 'pressure' in bars, 'XH2O_fl', 'XCO2_fl', and 'FluidProportion_wtper' (the proportion of the fluid in the system in wt%) if `verbose` is set to True). 

>If mutliple samples are passed as an ExcelFile object, a pandas DataFrame is returned with sample information plus calculated dissolved H$_2$O and CO$_2$ concentrations in the liquid, the fluid composition in mole fraction, and the proportion of the fluid in the system in wt%. Pressure (in bars) and Temperature (in $^{\circ}$C) columns are always returned.

In [1]:
import sys
sys.path.insert(0, '../')

import VESIcal as v

## For an entire dataset
### Import an Excel file

In [2]:
myfile = v.BatchFile('../../manuscript/example_data.xlsx')

### Do the calculation

In [3]:
dissolved = myfile.calculate_dissolved_volatiles(temperature=900.0, pressure=1000.0, X_fluid=0.5, print_status=True)
dissolved

myfile.save_csv('../tables/dissolved.csv', dissolved)

[=                   ] 7%  Working on sample BT-ex                            

[==                  ] 14%  Working on sample TVZMa-ex                            

[====                ] 21%  Working on sample TVZOh-ex                            

[=====               ] 29%  Working on sample Oh48-FTIR1-MI1-a                            

[=======             ] 36%  Working on sample Oh48-FTIR1-MI1-b                            

...

[===============     ] 79%  Working on sample Ma55-5a.1                            

[=================   ] 86%  Working on sample Ma57-3b.2                            

[==================  ] 93%  Working on sample Ma57-3c.1                            

[====================] 100%  Working on sample Ma57-3c.2                            


Saved ../tables/dissolved.csv


## For a single sample

### Extract a single sample from your dataset

In [4]:
SampleName = 'BT-ex'
extracted_bulk_comp = myfile.get_sample_composition(SampleName, asSampleClass=True)

### Do the calculation

In [5]:
v.calculate_dissolved_volatiles(sample=extracted_bulk_comp, temperature=900.0, pressure=2000.0, X_fluid=0.5).result

{'H2O_liq': 3.40551873148255, 'CO2_liq': 0.0704084969352978}

In [6]:
myfile.save_excel('dissolved.xlsx', calculations=[dissolved])

Saved dissolved.xlsx
